In [1]:
#### Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from time import time
import matplotlib.pyplot as plt
from operator import itemgetter

In [2]:
# Load training and test data into pandas dataframes
# ACCCESSING DATA FOLDER REFER THIS LINK 
# (https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python)

import os

#TO ACCESS THE PATH IN WHICH CURRENT SCRIPT IS RUNNING
fileDir = os.path.dirname(os.path.realpath('__file__'))

#For accessing the file inside a sibling folder.
#data_type = 'raw' or 'processed'
#country   = 'A' or 'B' or 'C'
#person_type = 'hhold' or 'indiv'
#dataset = 'test' or 'train'

def return_file_name(data_type = 'raw', country = 'A', person_type = 'hhold', dataset = 'train'):
    path = '../data/' + data_type + '/' + country + '/' + country + '_' + person_type + '_' + dataset +'.csv'
    print("Building path for file : (" + path +")")
    filename = os.path.join(fileDir, path)
    filename = os.path.abspath(os.path.realpath(filename))
    #print(filename + "\n")
    return filename

def return_train_test(data_type = 'raw', country = 'A', person_type = 'hhold'):
    train = pd.read_csv(return_file_name(data_type = data_type, country = country, person_type = person_type, dataset = 'train'))
    test = pd.read_csv(return_file_name(data_type = data_type, country = country, person_type = person_type, dataset = 'test'))
    # merge training and test sets into one dataframe
    full = pd.concat([train, test])
    return train,test,full

In [98]:
# Get size of dataframes
train_set,test_set,full_set = return_train_test(data_type = 'raw', country = 'A', person_type = 'indiv')

#shape[0] -> # of rows
#shape[1] -> # of cols
print("\nTrain Shape : " + str(train_set.shape))
print("Test Shape : " + str(test_set.shape))
print("Full Shape : " + str(full_set.shape) + "\n")

Building path for file : (../data/raw/A/A_indiv_train.csv)
Building path for file : (../data/raw/A/A_indiv_test.csv)

Train Shape : (37560, 44)
Test Shape : (18535, 43)
Full Shape : (56095, 44)



In [99]:
#Looking for Nans
#return a formatted percentage from a fraction
def percentage(numerator, denomenator):
    
    if type(numerator) == pd.core.series.Series:
        return (numerator/denomenator*100)
    
    elif type(numerator) == int or type(numerator) == float:
        return '{:.1f}%'.format(float(numerator)/float(denomenator)*100) 
    
    else:
        print("check type")

In [100]:
#Get percentage by variable of values which are not NaN
def return_bad_cols(dataframe):
    fill_precent_counts = percentage(dataframe.count()-1, dataframe.shape[0]-1)
    bad_cols = fill_precent_counts[fill_precent_counts < 100]
    return bad_cols

train_bad_cols = return_bad_cols(train_set)
test_bad_cols = return_bad_cols(test_set)
test_bad_cols

OdXpbPGJ    83.473616
dtype: float64

In [101]:
#make directory for asving bad columns statistics
# refer link (https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist)
stats_folder_path = '../data/stats/'

def make_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def create_stats_folder():
    for name in ['A', 'B', 'C']:
        foldername = os.path.join(fileDir, stats_folder_path + name + '/individual')
        foldername = os.path.abspath(os.path.realpath(foldername))
        make_dir(foldername)
        foldername = os.path.join(fileDir, stats_folder_path + name + '/household')
        foldername = os.path.abspath(os.path.realpath(foldername))
        make_dir(foldername)
        print("Trying to create folder for " + name)
        
def return_bad_col_dtype(bad_columns, dataframe):
    data_type = []
    for name in bad_columns:
        data_type.append(dataframe[name].dtype)
    return data_type

create_stats_folder()



Trying to create folder for A
Trying to create folder for B
Trying to create folder for C


In [102]:
def return_unique_values(bad_columns, dataframe):
    unique_vals = []
    for name in bad_columns:
        vals = dataframe[dataframe[name].notnull()][name].unique()
        unique_vals.append(vals.tolist())
    return unique_vals

def return_max_val_repeated(bad_columns, dataframe):
    freq_count_num = []
    val_colon_freq_precent = []
    for name in bad_columns:
        #refer (https://www.w3resource.com/python-exercises/numpy/python-numpy-exercise-94.php)
        unique_elements, counts_elements = np.unique(dataframe[dataframe[name].notnull()][name], return_counts=True)
        val_with_max_freq = unique_elements[np.argmax(counts_elements)]
        max_freq = int(np.max(counts_elements) / np.sum(counts_elements) * 100)
        dict_f = {val_with_max_freq : max_freq}
    
        dict_temp = {}
        for val, count in zip(unique_elements, counts_elements):
            dict_temp[val] = int(count / np.sum(counts_elements) * 100)
        
        freq_count_num.append(dict_f)
        val_colon_freq_precent.append(dict_temp)
    return freq_count_num, val_colon_freq_precent



In [103]:
#############################
##    data set analysis    ##
#############################

def data_set_analysis(bad_columns, dataset):
    #make a data frame 
    #refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
    bad_col_data = {'name': bad_columns.keys(), 'per_non_nan': bad_columns.values}
    bad_col_frame = pd.DataFrame(data=bad_col_data)
    
    #Add data type col
    bad_col_data_type = return_bad_col_dtype(bad_col_frame['name'], dataset)
    bad_col_frame['datatype'] = bad_col_data_type
    
    #unique vals col (not used)
    bad_col_unique_vals = return_unique_values(bad_col_frame['name'], dataset)
    #bad_col_frame['unique_vals'] = bad_col_unique_vals

    #show max freq count 
    bad_col_freq_count, temp = return_max_val_repeated(bad_col_frame['name'], dataset)
    bad_col_frame['freq_count'] = bad_col_freq_count
    bad_col_frame['unique_vals_colon_precent'] = temp

    return bad_col_frame
    
bad_frame_train = data_set_analysis(train_bad_cols, train_set)
bad_frame_test = data_set_analysis(test_bad_cols, test_set)
#make a data frame 
'''
#refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
train_bad_col_data = {'name': train_bad_cols.keys(), 'per_non_nan': train_bad_cols.values}
train_bad_col_frame = pd.DataFrame(data=train_bad_col_data)

#Add data type col
bad_col_data_type = return_bad_col_dtype(train_bad_col_frame['name'], train_set)
train_bad_col_frame['datatype'] = bad_col_data_type

#unique vals col (not used)
bad_col_unique_vals = return_unique_values(train_bad_col_frame['name'], train_set)
#train_bad_col_frame['unique_vals'] = bad_col_unique_vals

#show max freq count 
bad_col_freq_count, temp = return_max_val_repeated(train_bad_col_frame['name'], train_set)
train_bad_col_frame['freq_count'] = bad_col_freq_count
train_bad_col_frame['unique_vals_colon_precent'] = temp
'''

#train_set[train_set.FGWqGkmD.isnull()].groupby('poor')['poor'].count()
#train[train.FGWqGkmD.notnull()].groupby('poor')['poor'].count()

#for col_name, precent in zip(train_bad_cols.keys(),train_bad_cols.values):
    #print(col_name + " " + str(precent))

"\n#refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)\ntrain_bad_col_data = {'name': train_bad_cols.keys(), 'per_non_nan': train_bad_cols.values}\ntrain_bad_col_frame = pd.DataFrame(data=train_bad_col_data)\n\n#Add data type col\nbad_col_data_type = return_bad_col_dtype(train_bad_col_frame['name'], train_set)\ntrain_bad_col_frame['datatype'] = bad_col_data_type\n\n#unique vals col (not used)\nbad_col_unique_vals = return_unique_values(train_bad_col_frame['name'], train_set)\n#train_bad_col_frame['unique_vals'] = bad_col_unique_vals\n\n#show max freq count \nbad_col_freq_count, temp = return_max_val_repeated(train_bad_col_frame['name'], train_set)\ntrain_bad_col_frame['freq_count'] = bad_col_freq_count\ntrain_bad_col_frame['unique_vals_colon_precent'] = temp\n"

In [104]:
#country = 'A'
country = 'B'
#country = 'C'

type_people = 'household'
#type_people = 'individual'

stats_name_train = os.path.join(fileDir, stats_folder_path + country + '/'+ type_people + '/' + country +'_' + type_people + '_train.csv')
stats_name_train = os.path.abspath(os.path.realpath(stats_name_train))

stats_name_test = os.path.join(fileDir, stats_folder_path + country + '/'+ type_people + '/' + country +'_' + type_people + '_test.csv')
stats_name_test = os.path.abspath(os.path.realpath(stats_name_test))


bad_frame_train.to_csv(stats_name_train, index=False)

bad_frame_test.to_csv(stats_name_test, index=False)

In [105]:
train_bad_cols

OdXpbPGJ    83.31159
dtype: float64

In [106]:
test_bad_cols

OdXpbPGJ    83.473616
dtype: float64

In [107]:
col_float_64 = []
for i  in train_set.columns:
    if (train_set[i].dtype == 'float64'):
        col_float_64.append(i)
        #print(train_set[i].dtype)

len(col_float_64)

1

In [108]:
same_cols = []
for x,y in zip(col_float_64, train_bad_cols.keys()):
    if (x == y):
        same_cols.append(x)
print(same_cols)
len(same_cols)

['OdXpbPGJ']


1

In [109]:
a = train_set.describe()
names = ['count','mean', 'std', 'min', '25%', '50%', '75%', 'max']
    
a['name'] = names
a.to_csv("a_indiv_train.csv", index=False)


b = test_set.describe()
names = ['count','mean', 'std', 'min', '25%', '50%', '75%', 'max']
    
b['name'] = names
b.to_csv("a_indiv_test.csv", index=False)

In [47]:
a

,id,wJthinfa,ZvEApWrk,vuQrLzvK,FGWqGkmD,qrOrXLPM,BXOWgPgL,umkFMfvA,McFBIGsm,NjDdhqIe,...,IrxBnWxE,BRzuVmyf,dnlnKrAg,VyHofjLM,GrLBZowF,oszSdLhD,aAufyreG,cDhZjxaW,OSmfjCbE,IOMvIGQS
count,3255.000000,3255.000000,3255.000000,3255.000000,602.000000,3255.000000,2504.000000,890.000000,2504.000000,3255.000000,...,272.000000,1794.000000,532.000000,3255.000000,3255.000000,3255.000000,909.000000,3255.000000,2504.000000,3255.000000
mean,50057.635023,43.381260,96.040860,17.427343,-7.509967,22.203379,158.354633,-33.279775,301.106230,88.597849,...,0.647059,45.675585,-15965.135338,1.974808,-249.528111,0.670661,45.782178,-85.937020,-339.568291,78.568356
std,28615.901302,22.728441,105.556895,72.057949,9.499141,6.962658,124.535287,8.231694,155.904844,107.268927,...,9.097690,41.675286,39.715899,1.565015,322.468103,1.833827,49.499821,114.537914,147.833796,63.123421
min,17.000000,-126.000000,-2.000000,-125.000000,-53.000000,8.000000,-40.000000,-63.000000,-43.000000,-7.000000,...,-61.000000,9.000000,-16047.000000,-2.000000,-5044.000000,-23.000000,-6.000000,-3639.000000,-506.000000,0.000000
25%,25938.000000,26.000000,33.000000,-39.000000,-13.000000,16.000000,50.000000,-36.000000,185.000000,28.000000,...,3.000000,21.000000,-15999.000000,2.000000,-364.000000,1.000000,12.000000,-119.000000,-501.000000,50.000000
50%,50299.000000,42.000000,68.000000,27.000000,-8.000000,24.000000,150.000000,-36.000000,305.000000,63.000000,...,3.000000,36.000000,-15959.000000,2.000000,-184.000000,1.000000,39.000000,-59.000000,-356.000000,50.000000
75%,74848.000000,58.000000,138.000000,77.000000,2.000000,24.000000,250.000000,-27.000000,425.000000,98.000000,...,3.000000,51.000000,-15927.000000,2.000000,-64.000000,1.000000,48.000000,-39.000000,-256.000000,100.000000
max,99979.000000,122.000000,1069.000000,127.000000,2.000000,48.000000,500.000000,-18.000000,605.000000,1253.000000,...,3.000000,276.000000,-15911.000000,8.000000,-4.000000,1.000000,426.000000,1.000000,34.000000,900.000000


Index(['FGWqGkmD', 'BXOWgPgL', 'umkFMfvA', 'McFBIGsm', 'IrxBnWxE', 'BRzuVmyf',
       'dnlnKrAg', 'aAufyreG', 'OSmfjCbE'],
      dtype='object')

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')